### Dependencies & global variables


In [1]:
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import dask_geopandas
import geopandas as gpd
from shapely.geometry import Point
import os


county_fips = {
    'Cherokee': '13057',
    'Clayton': '13063',
    'Cobb': '13067',
    'DeKalb': '13089',
    'Douglas': '13097',
    'Fayette': '13113',
    'Forsyth': '13117',
    'Fulton': '13121',
    'Gwinnett': '13135',
    'Henry': '13151',
    'Rockdale': '13247'
}

fips_county = {v: k for k, v in county_fips.items()}

### Parcel file analysis - MR & Infutor


In [ ]:
# Convert ATTOM GeoJSON files to Geopackage

input_dir = 'Jetstream_Parcels/Metro'

for file in os.listdir(input_dir):
    if file.endswith('.geojson'):
        file_path = os.path.join(input_dir, file)
        county = fips_county[file.split('.')[0]]
        gdf = gpd.read_file(file_path)
        gdf = gdf[[
            'county',
            'apn',
            'apn2',
            'latitude',
            'longitude',
            'geometry'
        ]]
        gdf.to_file(f'Jetstream_Parcels/Metro/Geopackage/{county}.gpkg')
        print(f'export complete for {county}!')

In [ ]:
# read in MReeves parcel data first as Pandas dataframe
parcel_points = pd.read_csv(
    '5_Misc/MReeves_FAR/Eleven_County_FAR.csv', low_memory=False)

# remove parcels without any lat / long values
parcel_points = parcel_points[~parcel_points['LATITUDE'].isna()]

# convert DataFrame to Geodataframe
parcel_points = gpd.GeoDataFrame(
    parcel_points,
    geometry=gpd.points_from_xy(
        parcel_points['LONGITUDE'], parcel_points['LATITUDE']),
    crs="EPSG:4326"
).to_crs(epsg=26967)

# spatial join to Census tracts
atl_cts = gpd.read_file(
    '5_Misc/ARC_CTs.gpkg'
).to_crs(epsg=26967)

# pare down Census tracts
atl_cts = atl_cts[[
    'GEOID',
    'geometry'
]]

# spatial join
parcel_joined = gpd.sjoin(
    parcel_points,
    atl_cts,
    how='left',
    predicate='within'
).drop(columns='index_right')

# create a county name column
parcel_joined['county_name'] = parcel_joined['GEOID'].str[:5].map(fips_county)

In [ ]:
# join each county's gpkg file to the parcels
for county in county_fips.keys():
    county_gpkg = gpd.read_file(
        f'Jetstream_Parcels/Metro/Geopackage/{county}.gpkg')
    joined_gdf = gpd.sjoin(
        county_gpkg,
        parcel_points,
        how='left',
        predicate='contains'
    )
    joined_gdf.to_file(
        f'Jetstream_Parcels/Metro/Geopackage/{county}_joined.gpkg')
    print(f'export complete for {county} geopackage!')

In [169]:
# define dtype dictionary
dtype = {
    10: 'object',
    100: 'float64',
    101: 'float64',
    121: 'object',
    127: 'object',
    132: 'object',
    133: 'object',
    136: 'object',
    141: 'object',
    142: 'object',
    143: 'float64',
    144: 'float64',
    15: 'object',
    151: 'object',
    152: 'object',
    155: 'object',
    159: 'object',
    16: 'object',
    160: 'object',
    161: 'object',
    163: 'object',
    164: 'object',
    168: 'object',
    169: 'object',
    180: 'object',
    182: 'object',
    185: 'object',
    19: 'object',
    192: 'object',
    194: 'object',
    197: 'object',
    198: 'object',
    20: 'object',
    202: 'object',
    203: 'object',
    204: 'object',
    206: 'object',
    21: 'object',
    210: 'float64',
    212: 'float64',
    24: 'float64',
    28: 'float64',
    32: 'object',
    33: 'object',
    36: 'object',
    37: 'object',
    38: 'object',
    41: 'float64',
    45: 'float64',
    50: 'float64',
    52: 'object',
    54: 'object',
    55: 'object',
    57: 'object',
    58: 'object',
    59: 'float64',
    66: 'float64',
    69: 'object',
    70: 'object',
    75: 'object',
    77: 'object',
    79: 'float64',
    81: 'float64',
    85: 'float64',
    86: 'float64',
    88: 'float64',
    89: 'float64',
    96: 'object',
    97: 'object'}

# define filepath location
infutor_path = "/Users/willwright/Desktop/ARC/Research/ATTOM/6_Infutor/PROP_GA.txt"

# read in Infutor data
infutor_file = dd.read_csv(
    infutor_path,
    sep="\t",
    on_bad_lines='skip',
    low_memory=False,
    header=None,
    dtype='object',
)

# Assign temporary column names
infutor_file.columns = [f'col_{i}' for i in range(infutor_file.shape[1])]

In [175]:
# define the columns we'll need to pare down the huge initial dataframe
new_infutor = infutor_file[[
    'col_28',  # county code
    'col_57',  # latitude
    'col_58',  # longitude
    'col_59',  # LUC
    'col_101',  # land area (SF)
    'col_102'  # building size (SF)
]]

# rename columns to something sensible
new_infutor = new_infutor.rename(columns={
    'col_28': 'county_code',
    'col_57': 'lat',
    'col_58': 'long',
    'col_59': 'land_use',
    'col_101': 'area_land_SF',
    'col_102': 'area_bldg_SF'
})

# just get parcels in metro ATL
new_infutor['county_fips'] = '13' + new_infutor['county_code'].astype(str)
new_infutor['county_fips'] = new_infutor['county_fips'].str.split('.').str[0]
new_infutor = new_infutor[new_infutor['county_fips'].isin(
    county_fips.values())]
new_infutor = new_infutor.drop(columns=['county_code'])

# remove missing values from the lat / long columns
new_infutor = new_infutor.dropna(subset=['lat', 'long'])

# convert to pandas dataframe
new_infutor = new_infutor.compute()

# export to CSV
new_infutor.to_csv('4_DataExport/infutorATL.csv', index=False)
print('export complete!')

export complete!


### Assessor file for residential sales


In [ ]:
dtype = {
    'AreaBuilding': 'float64',
    'AreaGross': 'float64',
    'YearBuilt': 'float64',
    'PropertyUseStandardized': 'float64',
    'Area1stFloor': 'float64',
    'Area2ndFloor': 'float64',
    'AreaUpperFloors': 'float64'
}

# read in statewide file
assessor_file = dd.read_csv(
    '2_Assessor/ATLANTA_REGIONAL_COMMISSION_TAXASSESSOR_0002.txt',
    sep="\t",
    on_bad_lines='skip',
    low_memory=False,
    dtype=dtype
)

# only grab the columns we need
assessor_file = assessor_file[[
    '[ATTOM ID]',
    'PropertyAddressFull',
    'SitusCounty',
    'SitusStateCountyFIPS',
    'PropertyUseGroup',
    'PropertyUseStandardized',
    'AreaLotSF',
    'PropertyLatitude',
    'PropertyLongitude',
    'YearBuilt',
    'AreaBuilding',
    'AreaGross',
]]

# cast county FIPS as string variable
assessor_file['SitusStateCountyFIPS'] = assessor_file['SitusStateCountyFIPS'].astype(
    str)

# only include rows in ATL metro
assessor_file = assessor_file[assessor_file['SitusStateCountyFIPS'].isin(
    county_fips.values())]

# convert from Dask to Pandas
assessor_file = assessor_file.compute()

# just like with residential property, pare down the total assessor file
assessor_total = assessor_file[[
    '[ATTOM ID]',
    'PropertyAddressFull',
    'PropertyUseGroup',
    'AreaLotSF',
    'PropertyLatitude',
    'PropertyLongitude',
    'YearBuilt',
    'AreaBuilding',
    'AreaGross'
]]

# export the assessor file before splitting off the residential portions
assessor_total.to_csv('2_Assessor/ATL_assessor_file.csv', index=False)

# only want single-family residences in the dataset - from the ATTOM documentation
property_uses = {
    363: "BUNGALOW (RESIDENTIAL)",
    364: "CLUSTER HOME",
    366: "CONDOMINIUM",
    369: "DUPLEX (2 UNITS, ANY COMBINATION)",
    371: "MANUFACTURED, MODULAR, PRE-FABRICATED HOMES",
    373: "MOBILE HOME",
    378: "QUADPLEX (4 UNITS, ANY COMBINATION)",
    380: "RESIDENTIAL (GENERAL/SINGLE)",
    382: "ROW HOUSE",
    383: "RURAL RESIDENCE",
    385: "SINGLE FAMILY RESIDENCE",
    386: "TOWNHOUSE",
    388: "TRIPLEX (3 UNITS, ANY COMBINATION)"
}
assessor_residential = assessor_file[assessor_file['PropertyUseStandardized'].isin(
    property_uses.keys())]

# pare down the assessor file
assessor_residential = assessor_residential[[
    '[ATTOM ID]',
    'PropertyLatitude',
    'PropertyLongitude',
    'YearBuilt',
    'AreaBuilding'
]]

# create rows & columns variables for reporting
df_rows, df_columns = assessor_residential.shape[0], assessor_residential.shape[1]

print(f'Assessor rows: {df_rows:,}')
print(f'Assessor columns: {df_columns}')
assessor_residential.head(3)

### Recorder file - from URL


In [ ]:
dtype = {
    'ArmsLengthFlag': 'object',
    'Book': 'object',
    'DocumentNumberFormatted': 'object',
    'DocumentNumberLegacy': 'object',
    'DocumentRecordingCountyFIPs': 'object',
    'Grantee1NameSuffix': 'object',
    'Grantee2NameSuffix': 'object',
    'Grantee3InfoEntityClassification': 'object',
    'Grantee3NameFirst': 'object',
    'Grantee3NameFull': 'object',
    'Grantee3NameLast': 'object',
    'Grantee3NameMiddle': 'object',
    'Grantee3NameSuffix': 'object',
    'Grantee4InfoEntityClassification': 'object',
    'Grantee4NameFirst': 'object',
    'Grantee4NameFull': 'object',
    'Grantee4NameLast': 'object',
    'Grantee4NameMiddle': 'object',
    'Grantee4NameSuffix': 'object',
    'GranteeInfoEntityCount': 'float64',
    'GranteeMailAddressCRRT': 'object',
    'GranteeMailAddressCity': 'object',
    'GranteeMailAddressFull': 'object',
    'GranteeMailAddressHouseNumber': 'object',
    'GranteeMailAddressInfoFormat': 'object',
    'GranteeMailAddressState': 'object',
    'GranteeMailAddressStreetDirection': 'object',
    'GranteeMailAddressStreetName': 'object',
    'GranteeMailAddressStreetPostDirection': 'object',
    'GranteeMailAddressStreetSuffix': 'object',
    'GranteeMailAddressUnitPrefix': 'object',
    'GranteeMailAddressUnitValue': 'object',
    'GranteeMailCareOfName': 'object',
    'Grantor1NameMiddle': 'object',
    'Grantor1NameSuffix': 'object',
    'Grantor2InfoEntityClassification': 'object',
    'Grantor2InfoOwnerType': 'object',
    'Grantor2NameFirst': 'object',
    'Grantor2NameFull': 'object',
    'Grantor2NameLast': 'object',
    'Grantor2NameMiddle': 'object',
    'Grantor2NameSuffix': 'object',
    'Grantor3InfoEntityClassification': 'object',
    'Grantor3NameFirst': 'object',
    'Grantor3NameFull': 'object',
    'Grantor3NameLast': 'object',
    'Grantor3NameMiddle': 'object',
    'Grantor3NameSuffix': 'object',
    'Grantor4InfoEntityClassification': 'object',
    'Grantor4NameFirst': 'object',
    'Grantor4NameFull': 'object',
    'Grantor4NameLast': 'object',
    'Grantor4NameMiddle': 'object',
    'Grantor4NameSuffix': 'object',
    'GrantorAddressCity': 'object',
    'GrantorAddressFull': 'object',
    'GrantorAddressInfoFormat': 'object',
    'GrantorAddressState': 'object',
    'GrantorAddressUnitValue': 'object',
    'InstrumentNumber': 'object',
    'LegalDescriptionPart1': 'object',
    'LegalDescriptionPart2': 'object',
    'LegalDescriptionPart3': 'object',
    'LegalDistrict': 'object',
    'LegalLot': 'object',
    'LegalPlatMapBook': 'object',
    'LegalPlatMapPage': 'object',
    'LegalTract': 'object',
    'LegalUnit': 'object',
    'LegalDescriptionPart4': 'object',
    'LegalRange': 'object',
    'LegalTownship': 'object',
    'Mortgage1Amount': 'float64',
    'Mortgage1Book': 'object',
    'Mortgage1DocumentNumberFormatted': 'object',
    'Mortgage1DocumentNumberLegacy': 'object',
    'Mortgage1InterestMargin': 'float64',
    'Mortgage1LenderAddress': 'object',
    'Mortgage1LenderAddressCity': 'object',
    'Mortgage1LenderAddressState': 'object',
    'Mortgage1LenderCode': 'float64',
    'Mortgage1LenderNameFirst': 'object',
    'Mortgage1Page': 'object',
    'Mortgage1Term': 'float64',
    'Mortgage1Type': 'float64',
    'Mortgage2Amount': 'float64',
    'Mortgage2Book': 'object',
    'Mortgage2DocumentNumberFormatted': 'object',
    'Mortgage2DocumentNumberLegacy': 'object',
    'Mortgage2InstrumentNumber': 'object',
    'Mortgage2LenderAddress': 'object',
    'Mortgage2LenderAddressCity': 'object',
    'Mortgage2LenderAddressState': 'object',
    'Mortgage2LenderInfoEntityClassification': 'object',
    'Mortgage2LenderNameFirst': 'object',
    'Mortgage2LenderNameFullStandardized': 'object',
    'Mortgage2LenderNameLast': 'object',
    'Mortgage2RecordingDate': 'object',
    'Mortgage2TermDate': 'object',
    'Mortgage2TermType': 'object',
    'Page': 'object',
    'PropertyAddressHouseNumber': 'object',
    'PropertyAddressStreetDirection': 'object',
    'PropertyAddressStreetPostDirection': 'object',
    'PropertyAddressUnitPrefix': 'object',
    'PropertyAddressUnitValue': 'object',
    'PropertyAddressZIP': 'float64',
    'PropertyAddressZIP4': 'float64',
    'TitleCompanyStandardizedCode': 'float64',
    'TransferAmount': 'float64',
    'TransferAmountInfoAccuracy': 'float64',
    'TransferInfoDistressCircumstanceCode': 'float64',
    'TransferInfoPurchaseTypeCode': 'float64'
}

# read in statewide file
recorder_raw = dd.read_csv(
    '1_Recorder/ATLANTA_REGIONAL_COMMISSION_RECORDER_0003.txt',
    sep="\t",
    on_bad_lines='skip',
    low_memory=False,
    dtype=dtype
)

# merge to assessor file
recorder_joined = recorder_raw.merge(
    assessor_file,
    left_on='[ATTOM ID]',
    right_on='[ATTOM ID]'
)

# only include rows in ATL metro
recorder_joined = recorder_joined[recorder_joined['DocumentRecordingCountyFIPs'].isin(
    county_fips.values())]

# avoid SettingWithCopyWarning by first making a copy
recorder_joined = recorder_joined.copy()

# create year, month, year-month columns
recorder_joined['RecordingDate'] = dd.to_datetime(
    recorder_joined['RecordingDate'])
recorder_joined['year'] = recorder_joined['RecordingDate'].dt.year.astype(str)
recorder_joined['month'] = recorder_joined['RecordingDate'].dt.month.astype(
    str)
recorder_joined['year'] = recorder_joined['year'].astype(float)

# create new price per SF column
recorder_joined['price_sf'] = recorder_joined['TransferAmount'] / \
    recorder_joined['AreaBuilding']

# standard filters apply
recorder_joined = recorder_joined[
    (recorder_joined['TransferAmount'] > 0)
    & (recorder_joined['AreaBuilding'] > 0)
    & (recorder_joined['year'] >= 2020)
    & (recorder_joined['TransferInfoMultiParcelFlag'] != 7)
    & (recorder_joined['ArmsLengthFlag'] == "1")
]

# initial pare-down
recorder_joined = recorder_joined[[
    '[ATTOM ID]',
    'RecordingDate',
    'year',
    'month',
    'TransferAmount',
    'AreaBuilding',
    'price_sf',
    'YearBuilt',
    'PropertyLatitude',
    'PropertyLongitude'
]]

# convert to Pandas dataframe
recorder_joined = recorder_joined.compute()

recorder_joined['year'] = recorder_joined['year'].astype(
    float).round().astype(int).astype(str)
recorder_joined['month'] = recorder_joined['month'].astype(
    float).round().astype(int).astype(str)
recorder_joined['year-month'] = recorder_joined['year'] + \
    '-' + recorder_joined['month']

# rename & reorder columns
recorder_joined = recorder_joined.rename(columns={
    '[ATTOM ID]': 'ATTOM_ID',
    'RecordingDate': 'sale_date',
    'TransferAmount': 'sale_amt',
    'AreaBuilding': 'home_size',
    'YearBuilt': 'yr_blt',
    'PropertyLatitude': 'lat',
    'PropertyLongitude': 'long'
})

recorder_joined = recorder_joined[[
    'ATTOM_ID',
    'sale_date',
    'year',
    'month',
    'year-month',
    'sale_amt',
    'home_size',
    'price_sf',
    'yr_blt',
    'lat',
    'long'
]]

# export to CSV
recorder_joined.to_csv('1_Recorder/residentialSales.csv', index=False)

# create reporting variables for dataframe
df_rows, df_columns = recorder_joined.shape[0], recorder_joined.shape[1]

print(f'Recorder rows: {df_rows:,}')
print(f'Recorder columns: {df_columns}')
recorder_joined.head(5)

### sanity check - sales


In [ ]:
sales_23 = recorder_joined[recorder_joined['year'] == '2023']
sales_23_fulton = sales_23[sales_23['county'] == 'Fulton']
# sales_23_filtered = sales_23[(sales_23['price_sf'] > 5) & (
#     sales_23['price_sf'] < 800)]

sales_23_fulton = sales_23_fulton[[
    'ATTOM_ID',
    'address',
    'county',
    'sale_date',
    'sale_amt',
    'home_size',
    'price_sf'
]]

# print(f'filtered sales: {sales_23_filtered.shape[0]:,}')

# ax = sales_23_filtered['price_sf'].plot.kde(
#     ind=250,
#     title='Price / SF Distribution - Fulton Co.'
# )
# ax.xaxis.set_major_formatter('${x:,.0f}')

percentile_10 = sales_23_fulton['price_sf'].quantile(0.1)
percentile_90 = sales_23_fulton['price_sf'].quantile(0.9)
percentile_25 = sales_23_fulton['price_sf'].quantile(0.25)
percentile_75 = sales_23_fulton['price_sf'].quantile(0.75)
median_price = sales_23_fulton['price_sf'].median()
mean = sales_23_fulton['price_sf'].mean()
total_sales = sales_23_fulton.shape[0]
IQR_1090 = sales_23_fulton[(sales_23_fulton['price_sf'] >= percentile_10) & (
    sales_23_fulton['price_sf'] <= percentile_90)]
IQR_2575 = sales_23_fulton[(sales_23_fulton['price_sf'] >= percentile_25) & (
    sales_23_fulton['price_sf'] <= percentile_75)]

IQR_mean = IQR_1090['price_sf'].mean()
IQR_2575_mean = IQR_2575['price_sf'].mean()

print(f'10th percentile: ${percentile_10:.2f} per SF')
print(f'25th percentile: ${percentile_25:.2f} per SF')
print(f'MEDIAN: ${median_price:.2f} per SF')
print(f'MEAN: ${mean:.2f} per SF')
print(f'75th percentile: ${percentile_75:.2f} per SF')
print(f'90th percentile: ${percentile_90:.2f} per SF')
print('------')
print(f'total (unfiltered) sales: {total_sales:,}')
print('------')
print(f'sales in the IQR (25th-75th percentile range): {IQR_2575.shape[0]:,}')
print(f'sales in 10th-90th percentile range: {IQR_1090.shape[0]:,}')
print(f'10-90 mean: {IQR_mean}')
print(f'25-75 mean: {IQR_2575_mean}')

In [ ]:
ax = IQR_1090['price_sf'].plot.kde(
    ind=1000,
    title='Price / SF 10th-90th Percentiles (2023 Fulton County Sales)'
)
ax.xaxis.set_major_formatter('${x:,.0f}')

In [ ]:
ax = IQR_2575['price_sf'].plot.kde(
    ind=1000,
    title='Price / SF 25th-75th Percentiles (2023 Fulton County Sales)'
)
ax.xaxis.set_major_formatter('${x:,.0f}')

In [ ]:
percentile_25 = sales_23['price_sf'].quantile(0.25)
percentile_75 = sales_23['price_sf'].quantile(0.75)
percentile_10 = sales_23['price_sf'].quantile(0.1)
percentile_90 = sales_23['price_sf'].quantile(0.9)

IQR_df = sales_23[(sales_23['price_sf'] >= percentile_25) & (
    sales_23['price_sf'] <= percentile_75)]
df_1090 = sales_23[(sales_23['price_sf'] >= percentile_10) & (
    sales_23['price_sf'] <= percentile_90)]

print(f'total sales in 2023: {sales_23.shape[0]:,}')
print(f'total sales in 2023 in IQR: {IQR_df.shape[0]:,}')

IQR_df.to_csv('4_DataExport/IQR_df.csv', index=False)
df_1090.to_csv('4_DataExport/df_1090.csv', index=False)

### Xref - from URL


In [ ]:
# read in statewide file
xref_file = pd.read_csv(
    '3_PropertyMatch/ATLANTA_REGIONAL_COMMISSION_XREF_PROPERTYTOBOUNDARYMATCH_PARCEL_0002.txt',
    sep="\t",
    on_bad_lines='skip',
    low_memory=False
)

# # only include rows in ATL metro
# xref_file = xref_file[xref_file['DocumentRecordingCountyFIPs'].isin(
#     atl_fips)]

print(f'Assessor rows: {xref_file.shape[0]:,}')
print(f'Assessor columns: {xref_file.shape[1]}')
xref_file.head()

### Export county-specific sales


In [ ]:
# define county list
county_list = [
    'Gwinnett',
    'Cherokee',
    'DeKalb',
    'Clayton',
    # 'Cobb',
    # 'Douglas',
    # 'Fayette',
    # 'Forsyth',
    # 'Fulton',
    # 'Henry',
    # 'Rockdale'
]

# loop through and export county-specific files
for county in county_list:
    df = recorder_joined[recorder_joined['county'] == county]
    df = df.drop(columns='county')
    df.to_csv(f'4_DataExport/{county}Sales.csv', index=False)
    print(f'successfully exported sales for {county} County!')